# Target smoothing using Wednesday Denoiser Algorithm 


Case: Training n=250 run to assess Wednesday target smoothing algorithm using the Wednesday Noisy-ML Simulation Framework 

Part of a paper submission: *The Invisible Performance of Regression Models on Noisy Measurements*

Author: Fatma-Elzahraa Eid, Broad Institute of MIT and Harvard 

In [1]:
# [0] Imports 
import os
import numpy as np
import pandas as pd
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # '2' to suppress TensorFlow informational message (oneDNN)
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
from scipy.stats import shapiro, anderson
import warnings

In [2]:

# [1] Simulation Class
class NoiseSimulation:
    """
    Class for performing noise simulation on a neural network model.
    Encapsulates all relevant functions for data generation, training, and evaluation.
    """
    def __init__(self, results_folder='results', results_filename='Wednesday_results_temp.csv',
                 n_rounds=100, 
                 features_range=[10,500],
                 example_factor=200,
                 noise_factor_range=[0.01, 2.0], 
                 smoothing_factor=0.5, 
                 smoothing_iterations=10):
        """
        Initialize the NoiseSimulation class with necessary parameters.
        
        :param results_folder: Folder to save results
        :param results_filename: Name of the results CSV file
        :param n_rounds: Number of simulation rounds
        :param noise_factor_range: Range for noise factor
        :param smoothing_factor: Smoothing factor for label smoothing, 0.5 for averaging 
        :param smoothing_iterations: Number of smoothing iterations
        """
        self.results_folder = results_folder
        self.results_filename = results_filename
        self.n_rounds = n_rounds
        self.features_range = features_range
        self.example_factor = example_factor
        self.noise_factor_range = noise_factor_range
       
        self.smoothing_factor = smoothing_factor
        self.smoothing_iterations = smoothing_iterations
        self.results = {
            'n_features': [], 'n_samples': [], 'corr_Ytrue_Ynoisy1': [], 'corr_Ytrue_Ynoisy2': [],
            'corr_Ynoisy1_Ynoisy2': [], 'corr_pred0_Ytrue': [], 'corr_pred_Ytrue': [], 'corr_pred_Ynoisy1': [],
            'corr_pred_Ynoisy2': [],
            'iteration_corr_pred_Ytrue': [], 'iteration_corr_pred_Ynoisy1': [], 'corr_pred_Ytrue_optimal': []
        }

        # Create the results folder if it does not exist
        if not os.path.exists(self.results_folder):
            os.makedirs(self.results_folder)
        self.results_filename_infolder = os.path.join(self.results_folder, self.results_filename)

    @staticmethod
    def generate_features(n_features=10, n_samples=1000):
        """
        Generate features using various statistical distributions.
        
        :param n_samples: Number of samples to generate
        :param n_features: Number of features to generate
        :return: A 2D NumPy array with generated features
        """
        distribution_pool = [
            lambda: np.random.normal(0, 1, n_samples),
            lambda: np.random.uniform(-1, 1, n_samples),
            lambda: np.random.lognormal(0, 1, n_samples),
            lambda: np.random.exponential(1, n_samples),
            lambda: np.random.beta(2, 5, n_samples),
            lambda: np.random.gamma(2, 2, n_samples)
        ]
        features = [np.random.choice(distribution_pool)() for _ in range(n_features)]
        return np.column_stack(features)

    @staticmethod
    def generate_targets(X):
        """
        Generate target values based on transformations of standardized features.
        
        :param X: Standardized feature matrix
        :return: Generated target values
        """
        expressions_pool = [
            lambda x: np.random.uniform(1.0, 2.0, 1)[0] * np.sin(x),
            lambda x: np.log(np.abs(x) + np.random.uniform(0.01, 2.0, 1)[0]),
            lambda x: x ** 2,
            lambda x: x ** 3,
            lambda x: np.sqrt(np.abs(x)),
            lambda x: np.exp(x / (np.abs(x) + np.random.uniform(0.01, 2.0, 1)[0]))
        ]
        terms = [np.random.choice(expressions_pool) for _ in range(X.shape[1])]
        transformed_columns = [term(X[:, i]) for i, term in enumerate(terms)]
        return sum(transformed_columns)

    @staticmethod
    def noise_factor(Y, noise_factor_range=[0.05, 0.1]):
        """
        Determine noise factor based on the standard deviation of target values.
        
        :param Y: Target values
        :param noise_factor_range: Range for noise factor
        :return: Calculated noise level
        """
        variability = np.std(Y)
        noise_factor = np.random.uniform(noise_factor_range[0], noise_factor_range[1])
        noise_level = noise_factor * variability
        return noise_level

    @staticmethod
    def add_noise(Y, noise_level):
        """
        Add Gaussian noise to target values.
        
        :param Y: Target values
        :param noise_level: Standard deviation of the Gaussian noise
        :return: Noisy target values
        """
        return Y + np.random.normal(0, noise_level, Y.shape[0])

    @staticmethod
    def norm_y(Y):
        """
        Normalize target values to ensure normal distribution using log transformation.
        
        :param Y: Target values
        :return: Normalized target values
        """
        warnings.filterwarnings("ignore", message="p-value may not be accurate for N > 5000.")
        _, p_value = shapiro(Y)
        alpha = 0.05
        if p_value < alpha:
            if np.min(Y) <= 0:
                Y = Y - np.min(Y) + 1
            Y_transformed = np.log(Y)
            return Y_transformed
        else:
            return Y

    @staticmethod
    def build_model(input_shape):
        """
        Build and compile a neural network model.
        
        :param input_shape: Shape of the input layer
        :return: Compiled Keras model
        """
        model = keras.Sequential([
            keras.layers.Dense(16, activation='relu', input_shape=input_shape),
            keras.layers.Dense(8, activation='relu'),
            keras.layers.Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        return model

    def run_simulation(self, n_samples=1000, n_features=10, noise_factor_range=[0.01, 2.0]):
        """
        Run a single round of noise simulation.
        
        :param n_samples: Number of samples to generate
        :param n_features: Number of features to generate
        :param noise_factor_range: Range for general noise factor
        :return: Correlation results
        """
        X = self.generate_features(n_features, n_samples)
        X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        X_stacked = np.vstack((X_train_scaled, X_test_scaled))
        Y_stacked = self.generate_targets(X_stacked)
        Y_stacked = self.norm_y(Y_stacked)
        Ytrue_train, Ytrue_test = np.split(Y_stacked, [X_train_scaled.shape[0]], axis=0)
        noise_level1 = self.noise_factor(Ytrue_train, noise_factor_range)
        noise_level2 = self.noise_factor(Ytrue_train, noise_factor_range)
        Y_replicate1 = self.add_noise(Y_stacked, noise_level1)
        Y_replicate2 = self.add_noise(Y_stacked, noise_level2)
        Y_replicate1_train, Y_replicate1_test = np.split(Y_replicate1, [X_train_scaled.shape[0]], axis=0)
        Y_replicate2_train, Y_replicate2_test = np.split(Y_replicate2, [X_train_scaled.shape[0]], axis=0)
        corr_Ytrue_Ynoisy1 = pearsonr(Y_replicate1_train, Ytrue_train)[0]
        corr_Ytrue_Ynoisy2 = pearsonr(Y_replicate2_train, Ytrue_train)[0]
        corr_Ynoisy1_Ynoisy2 = pearsonr(Y_replicate1_train, Y_replicate2_train)[0]

        # Train model on Ytrue and test on Ytrue for optimal performance
        optimal_model = self.build_model((n_features,))
        early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
        optimal_model.fit(X_train_scaled, Ytrue_train, epochs=1000, validation_split=0.2,
                          callbacks=[early_stopping], verbose=0)
        Y_pred_optimal = optimal_model.predict(X_test_scaled).flatten()
        corr_pred_Ytrue_optimal = pearsonr(np.array(Ytrue_test).flatten(), Y_pred_optimal)[0]

        #model1 = self.build_model((n_features,))
        #model2 = self.build_model((n_features,))
        model = self.build_model((n_features,))
        model.fit(X_train_scaled, Y_replicate1_train, epochs=1000, validation_split=0.2,
                  callbacks=[early_stopping], verbose=0)
        Y_pred = model.predict(X_test_scaled).flatten()
        corr_pred0_Ytrue = pearsonr(np.array(Ytrue_test).flatten(), Y_pred)[0]
        
        
        # Wednesday Smoothing
        """
        Input: dataset (X,Ynoisy), where X are features describing an entity, data point, or interactions, Ynoisy is noisy measurements subject to some Gaussian noise. 
        - Let Ystart = Ynoisy
        - Build model on (X,Ystart) 
        - Use the model to predict on X to generate Ypred.
        - Use Equation (Eq1) to assess the true quality of Ystart to see if it needs to be improved
        - If the quality of Ystart needs to be improved (say 0.9), Use Equation (Eq1) 
        to assess whether the quality of Ypred is better than that of Ystart. 
        - If Quality(Ypred)>YQualtiy(Ystart), Ystart= average(Ystart,Ypred)
        - Repeat Steps 3 to 7 until Step 6 condition is not true

        Output: Ystart as the final output and also the starting quality and end quality 

        Eq1: Corr(Ynoisy,Ypred) = sqrt(Cov(Ypred,Ynoisy)/Var(Ypred)) 
        """
        
        iteration_corr_pred_Ytrue = []
        iteration_corr_pred_Ynoisy1 = []
        Y_start = Y_replicate1_train.copy()
        
        for _ in range(self.smoothing_iterations):
            
            # Predict  
            Y_pred = model.predict(X_train_scaled).flatten()
            
            # ---- Convergence Test 
            # Ystart quality 
            A = np.sqrt( np.cov(np.stack((Y_start, Y_pred), axis=0))[0,1])
            B = np.sqrt(np.var(Y_start))
            Y_start_quality =np.divide( A ,  B ) 
            
            # Quality of averaged Ystart,Ypred
            A = np.sqrt( np.cov(np.stack((Y_start, Y_pred), axis=0))[0,1])
            B = np.sqrt(np.var(Y_pred))
            Y_pred_quality =np.divide( A , B ) 
           
            # Test: if new_quality > old_quality, update; else, exist        
            
            
            if   Y_pred_quality >   Y_start_quality:   
                                               
                iteration_corr_pred_Ytrue.append(pearsonr(np.array(Ytrue_train).flatten(), Y_pred)[0])
                iteration_corr_pred_Ynoisy1.append(pearsonr(np.array(Y_replicate1_train).flatten(), Y_pred)[0])
                
                # Update
                Y_start = self.smoothing_factor * Y_start + (1 - self.smoothing_factor) * Y_pred

                # Update                             
                #Y_start = 0.5 * (Y_start + Y_pred)        
                                               
                # New training round 
                model = self.build_model((n_features,))
                model.fit(X_train_scaled, Y_start, epochs=1000, validation_split=0.2,
                          callbacks=[early_stopping], verbose=0)
                                               
                


            else:
                # Update after smoothing 
                Y_pred_test = model.predict(X_test_scaled).flatten()
                
                # Report current iteration qualities 
                corr_pred_Ytrue = pearsonr(np.array(Ytrue_test).flatten(), Y_pred_test)[0]
                corr_pred_Ynoisy1 = pearsonr(np.array(Y_replicate1_test).flatten(), Y_pred_test)[0]
                corr_pred_Ynoisy2 = pearsonr(np.array(Y_replicate2_test).flatten(), Y_pred_test)[0] 
           
                
                break 
            
            # In case Iterations are done but converagence did not happen, to avoid  
            # Report current iteration qualities 
            Y_pred_test = model.predict(X_test_scaled).flatten()
            corr_pred_Ytrue = pearsonr(np.array(Ytrue_test).flatten(), Y_pred_test)[0]
            corr_pred_Ynoisy1 = pearsonr(np.array(Y_replicate1_test).flatten(), Y_pred_test)[0]
            corr_pred_Ynoisy2 = pearsonr(np.array(Y_replicate2_test).flatten(), Y_pred_test)[0] 
       


                                               
        return (corr_Ytrue_Ynoisy1, corr_Ytrue_Ynoisy2, corr_Ynoisy1_Ynoisy2,
                corr_pred0_Ytrue, corr_pred_Ytrue, corr_pred_Ynoisy1, corr_pred_Ynoisy2,
                iteration_corr_pred_Ytrue, iteration_corr_pred_Ynoisy1, corr_pred_Ytrue_optimal)

 
    
    
    def run(self):
        """
        Run the simulation for a specified number of rounds.
        """
        for _ in range(self.n_rounds):
            n_features = np.random.randint(self.features_range[0], self.features_range[1])
            n_samples = n_features * self.example_factor
            (corr_Ytrue_Ynoisy1, corr_Ytrue_Ynoisy2, corr_Ynoisy1_Ynoisy2,
             corr_pred0_Ytrue, corr_pred_Ytrue, corr_pred_Ynoisy1, corr_pred_Ynoisy2,
             iteration_corr_pred_Ytrue, iteration_corr_pred_Ynoisy1, corr_pred_Ytrue_optimal) = self.run_simulation(
                n_samples=n_samples, n_features=n_features,
                noise_factor_range=self.noise_factor_range,
            )
            self.results['n_features'].append(n_features)
            self.results['n_samples'].append(n_samples)
            self.results['corr_Ytrue_Ynoisy1'].append(corr_Ytrue_Ynoisy1)
            self.results['corr_Ytrue_Ynoisy2'].append(corr_Ytrue_Ynoisy2)
            self.results['corr_Ynoisy1_Ynoisy2'].append(corr_Ynoisy1_Ynoisy2)
            self.results['corr_pred0_Ytrue'].append(corr_pred0_Ytrue)
            self.results['corr_pred_Ytrue'].append(corr_pred_Ytrue)
            self.results['corr_pred_Ynoisy1'].append(corr_pred_Ynoisy1)
            self.results['corr_pred_Ynoisy2'].append(corr_pred_Ynoisy2)
            self.results['iteration_corr_pred_Ytrue'].append(iteration_corr_pred_Ytrue)
            self.results['iteration_corr_pred_Ynoisy1'].append(iteration_corr_pred_Ynoisy1)
            self.results['corr_pred_Ytrue_optimal'].append(corr_pred_Ytrue_optimal)

            # Check the lengths of each array in self.results
            lengths = {key: len(value) for key, value in self.results.items()}
            print(lengths)  # Print the lengths to debug

            # Check if all arrays have the same length before creating DataFrame
            array_lengths = [len(arr) for arr in self.results.values()]
            if len(set(array_lengths)) != 1:
                raise ValueError(f"All arrays must be of the same length. Current lengths: {array_lengths}")

            # Save after each iteration, do not wait till the end     
            results_df = pd.DataFrame(self.results)
            results_df.to_csv(self.results_filename_infolder, index=False)

In [3]:
# [2] Run the Simulation
simulation = NoiseSimulation(results_folder='results', 
                             results_filename='Wednesday_TargetSmoothing_n250.csv',
                             n_rounds=1000, 
                             features_range=[10,100],
                             example_factor=200,
                             noise_factor_range=[0.01, 2.0],
                             smoothing_factor=0.5, 
                             smoothing_iterations=20)
simulation.run()


112/112 [==============================] - 0s 2ms/step
{'n_features': 1, 'n_samples': 1, 'corr_Ytrue_Ynoisy1': 1, 'corr_Ytrue_Ynoisy2': 1, 'corr_Ynoisy1_Ynoisy2': 1, 'corr_pred0_Ytrue': 1, 'corr_pred_Ytrue': 1, 'corr_pred_Ynoisy1': 1, 'corr_pred_Ynoisy2': 1, 'iteration_corr_pred_Ytrue': 1, 'iteration_corr_pred_Ynoisy1': 1, 'corr_pred_Ytrue_optimal': 1}
75/75 [==============================] - 0s 862us/step
{'n_features': 2, 'n_samples': 2, 'corr_Ytrue_Ynoisy1': 2, 'corr_Ytrue_Ynoisy2': 2, 'corr_Ynoisy1_Ynoisy2': 2, 'corr_pred0_Ytrue': 2, 'corr_pred_Ytrue': 2, 'corr_pred_Ynoisy1': 2, 'corr_pred_Ynoisy2': 2, 'iteration_corr_pred_Ytrue': 2, 'iteration_corr_pred_Ynoisy1': 2, 'corr_pred_Ytrue_optimal': 2}
63/63 [==============================] - 0s 863us/step
{'n_features': 3, 'n_samples': 3, 'corr_Ytrue_Ynoisy1': 3, 'corr_Ytrue_Ynoisy2': 3, 'corr_Ynoisy1_Ynoisy2': 3, 'corr_pred0_Ytrue': 3, 'corr_pred_Ytrue': 3, 'corr_pred_Ynoisy1': 3, 'corr_pred_Ynoisy2': 3, 'iteration_corr_pred_Ytrue': 3,


KeyboardInterrupt

